In [ ]:
import pandas as pd
import re
import numpy as np
! pip install openpyxl

In [ ]:
sla = pd.read_excel(r'../input/shopee-code-league-20/_DA_Logistics/SLA_matrix.xlsx',engine = 'openpyxl')
orders = pd.read_csv(r'../input/shopee-code-league-20/_DA_Logistics/delivery_orders_march.csv')

In [ ]:
sla
#看起來很奇怪，不過從表中，大概可以猜出是一個對照表，而且index是出發地(from)，column是目的地(to)，然後表裡面是運送時間
#他的columns看起來是自動建立的，應該是要第一個row拿來做column才對
#row 6-8看起來像是其他資訊，而且只有第一個column有東西，應該是註解之類的，不過要把它祥細看一下
#看起來column:'Unnamed: 1'才是我要的index
#row:0才是我要的column

In [ ]:
for i in sla.iloc[:,0]:
    print(i)
#確實row 6-8只是註解而已

In [ ]:
#看起來column:'Unnamed: 1'才是我要的index
sla = sla.set_index('Unnamed: 1')
#row:0才是我要的column
sla.columns = sla.iloc[0]
#設定表格index和column的name，這樣比較看得懂
sla.index.name = 'from'
sla.columns.name='to'
display(sla)
#把多的內容拿掉
sla = sla.iloc[1:5,1:5]
display(sla)
#把表格中的string轉換成int
sla = sla.applymap(lambda x:int(re.match('^(\d+) working days$',x).group(1)))
display(sla)
display(sla.dtypes)
#把sla的index和column內容都變小寫
sla.index = sla.index.map(lambda x:x.lower())
sla.columns = sla.columns.map(lambda x:x.lower())
display(sla)
#因為沒有後面重複的地點，所以只用最後一個字
sla.index = sla.index.map(lambda x:x.split()[-1])
sla.columns = sla.columns.map(lambda x:x.split()[-1])
display(sla)
#到這邊sla整理完成了

In [ ]:
orders.head()

In [ ]:
#先整理時間格式
#以下是範的應對關係
#1583137548 (Converted to 2020-03-02 4:25:48 PM Local Time)
print(pd.to_datetime(1583137548,unit='s') + pd.Timedelta(hours=8))
#1583733540 (Converted to 2020-03-09 1:59:00 PM Local Time)
print(pd.to_datetime(1583733540,unit='s') + pd.Timedelta(hours=8))
#確定這樣轉換是對的，套用到資料上
#因為時區都是用同一個時區，所以這邊就直接暴力加8小時，不另外設定時區
orders['pick'] = pd.to_datetime(orders['pick'],unit='s') + pd.Timedelta(hours=8)
orders['1st_deliver_attempt'] = pd.to_datetime(orders['1st_deliver_attempt'],unit='s') + pd.Timedelta(hours=8)
orders['2nd_deliver_attempt'] = pd.to_datetime(orders['2nd_deliver_attempt'],unit='s') + pd.Timedelta(hours=8)
display(orders.head())
#因為題目只看日期，不看時間，把詳細時間丟掉
orders[['pick','1st_deliver_attempt','2nd_deliver_attempt']] = orders[['pick','1st_deliver_attempt','2nd_deliver_attempt']].applymap(lambda x: x.date())
orders[['pick','1st_deliver_attempt','2nd_deliver_attempt']] = orders[['pick','1st_deliver_attempt','2nd_deliver_attempt']].astype('datetime64')#把datatype變回pd datetime object
display(orders.head())
#整理一下欄位名稱，這樣比較好打
orders.columns = ['orderid','pick','st','nd','to','from']
display(orders.head())
#整理地址，變成全小寫
orders['to'] = orders['to'].map(lambda x:x.lower())
orders['from'] = orders['from'].map(lambda x:x.lower())
display(orders.head())

In [ ]:
#這邊可以證明地址的最後一個字都是我們sla裡面有的地點
t = orders['to'].map(lambda x: x.split()[-1])
f = orders['from'].map(lambda x: x.split()[-1])
print(t.isin(sla.columns).all())#sla的columns是to
print(f.isin(sla.index).all())#sla的index是from

In [ ]:
#只留下要的地名
orders['to'] = orders['to'].map(lambda x: x.split()[-1])
orders['from'] = orders['from'].map(lambda x: x.split()[-1])
display(orders.head())

In [ ]:
#計算時間距離
#製作一張表，利用查詢可以把所有日期都映射到int，因為假日的部分不計，所以數字不會增加，
#這邊看不懂沒關係，直接往下拉，看display出來是甚麼東西就會懂了

#要先知道製作的區間，所以找到整個dataset的日期最大值及最小值，才有辦法知道起訖日期
max_date = orders[['pick','st','nd']].max().max()
min_date = orders[['pick','st','nd']].min().min()

all_dates = pd.date_range(min_date,max_date)#包含最後一個日期

#找出星期日
sunday = all_dates[all_dates.weekday == 6]
#登記國定假日
holiday = pd.DatetimeIndex(['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
# 2020-03-08 (Sunday);
# 2020-03-25 (Wednesday);
# 2020-03-30 (Monday);
# 2020-03-31 (Tuesday)

#找出區間內所有不用上班的日子
day_off = sunday.union(holiday)

#開始製作表
date_delta = pd.Series(index = all_dates,dtype=int)
#先從0開始計算
delta_value = 0
for date in all_dates:
    if not date in day_off:
        #只有工作天的日子才會+1
        delta_value += 1
    #紀錄這天的delta value
    date_delta[date] = delta_value
    
display(date_delta)

#計算工作日的方法
def calaulate_working_days(start,end):
    #因為nd有可能有nat所以要另外處理
    if pd.isnull(end):
        return np.nan
    return date_delta[end] - date_delta[start]

#下面這個計算時間的方法，理論上可行，可是跑超久，跑不出來，所以我就放棄了
# holiday = pd.DatetimeIndex(['2020-03-08','2020-03-25','2020-03-30','2020-03-31'])
# def calaulate_working_days(start,end):
#     if pd.isnull(end):
#         return np.nan
#     all_days = pd.date_range(start,end)
#     all_days = all_days.difference(all_days[[0]])
#     all_days = all_days[all_days.weekday != 6]
#     all_days = all_days.difference(holiday)
#     return all_days.size

# orders['st_delta'] = orders.apply(lambda df: calaulate_working_days(*df[['pick','st']]),axis=1)
# orders['nd_delta'] = orders.apply(lambda df: calaulate_working_days(*df[['st','nd']]),axis=1)

In [ ]:
#把實際日期抽換成delta_value
#用replace比用apply花較少時間，所以用replace
orders['pick_value'] = orders['pick'].replace(date_delta)
orders['st_value'] = orders['st'].replace(date_delta)
orders['nd_value'] = orders['nd'].replace(date_delta)
#利用delta_value相減求得兩個事件之間實際距離幾個工作天
orders['st_delta'] = orders['st_value'] - orders['pick_value'] #從pick到first_attempt
orders['nd_delta'] = orders['nd_value'] - orders['st_value'] #從first_attempt到second_attempt

display(orders.head())

In [ ]:
#因為要用replace減少運算時間，所以sla要改成能被replace使用的樣子
stack_sla = sla.stack()
display(stack_sla)
stack_sla.index = stack_sla.index.map(lambda x: ','.join(x))
display(stack_sla)

In [ ]:
#新增一個欄位，能夠利用stack_sla，做replace
orders['route'] = orders['from'] +','+ orders['to']
#用replace取代成送貨時間
orders['transport'] = orders['route'].replace(stack_sla)

display(orders.head())

In [ ]:
#依照定義，開始記錄哪些有late
orders['st_late'] = 0
orders['nd_late'] = 0
orders['is_late'] = 0

#first_attempt_time > sla_matrix ---> st_late
orders.loc[orders['st_delta']>orders['transport'],'st_late'] = 1
#second_attempt - first_attempt > 3 ---> nd_late
orders.loc[orders['nd_delta']>3,'nd_late'] = 1

#first_attempt_time > sla_matrix ---> is_late
orders.loc[orders['st_delta']>orders['transport'],'is_late'] = 1
#second_attempt - first_attempt > 3 ---> is_late
orders.loc[orders['nd_delta']>3,'is_late'] = 1

In [ ]:
display(orders.query("(st_late==0)&(nd_late==1)").head(20))
display(orders.query("(st_late==1)&(nd_late==0)").head(20))
display(orders.query("(st_late==1)&(nd_late==1)").head(20))
display(orders.query("(st_late==0)&(nd_late==0)").head(20))

In [ ]:
orders[['st_late','nd_late','orderid']].groupby(['st_late','nd_late']).agg('count')

In [ ]:
answer = orders[['orderid','is_late']]
print(answer['is_late'].value_counts())
answer